This script serves to obtain the timing information for first level modeling. ** it is run locally as it takes in the raw fMRI data

In [1]:
import pandas as pd
import os

In [2]:
os.getcwd()

'/Users/jackiebeltran/Documents/RL/scripts'

In [11]:
flist=os.listdir("../raw_data_N83")

In [13]:
print("Participants in this dataset are: " + str(len(flist)))

Participants in this dataset are: 83


In [14]:
dataDir='../raw_data_N83/'
resDir='../data/regressor/'

In [16]:
def regressors(fname):
#find user id
    fvals=fname.split('_')
    uid=fvals[0] 
    
    print(f'Starting Analysis for {uid}')
#load df
    df=pd.read_csv(dataDir+fname)
    
# trigger val
    trig=df.trigger[0]
    
#Fix values
    fix=df.fix_onset.apply(lambda x: x-trig)
    fix.to_csv(f'{resDir+uid}_fix.txt',index=False,header=False)
    print(f'{uid} - Saved Fix values')
    
#Stimulus values
    df = df.assign(Stim = lambda x: (x['stim_onset'] -trig))
    cond=list(df.condition.unique())
    for i in cond:
        df_temp=df.loc[(df['condition']== i)]
        df_temp.Stim.to_csv(f'{resDir+uid}_stimulus_{i}.txt',index=False,header=False)
    print(f'{uid} - Saved Stimulus values')
    
#Choice values
    df['goNogo']=df["key_resp_2.keys"].replace({"None": "Nogo", "2":"go"})
    
    df = df.assign(Choice = lambda x: (x['choice_onset'] -trig))
    df.Choice.to_csv(f'{resDir+uid}_choice.txt',index=False,header=False)
        
    print(f'{uid} - Saved Choices values')
        
#Feedback
    df = df.assign(FedOn = lambda x: (x['outcome_onset'] -trig))
    
    # save all feedback info in one file 
    df.FedOn.to_csv(f'{resDir+uid}_feedback.txt',index=False,header=False)
    
    print(f'{uid} - Saved all Feedback values')
    
    condFed=list(df.feedback.unique())   
    for i in condFed:
        df_temp=df.loc[(df['feedback']== i)]
        df_temp.FedOn.to_csv(f'{resDir+uid}_feedback_{i}.txt',index=False,header=False)
    
    print(f'{uid} - Saved Feedback values')
    
#Condition-Feedback
    condChoice=list(df.goNogo.unique())
    condFed=[('gain','win'), 
             ('gain','nothing'),
             ('loss','loss'),
             ('loss','nothing'),
            ('neutral','nil')]
    
    for c, f in condFed:
        df_temp=df.loc[(df['condition']== c) & (df['feedback']== f)]
        if len(df_temp):
            df_temp.FedOn.to_csv(f'{resDir+uid}_feedback_{c}-{f}.txt',index=False,header=False)

    print(f'{uid} - Saved Condition-Feedback values')
    
#KeyResp
    df['KeyRT']=None
    for i in range(len(df)):
        if df.goNogo[i]=='No go':
            df.KeyRT[i]=0
        elif df.goNogo[i]=='go':
            df.KeyRT[i]=float(df['key_resp_2.rt'][i])
    
    df = df.assign(KeyOn = lambda x: x['Stim'] + x['KeyRT'])
    
    for i in condChoice:
        if i=='go':
            df_temp=df.loc[(df['goNogo']== i)]
            df_temp.KeyOn.to_csv(f'{resDir+uid}_KeyRT_{i}.txt',index=False,header=False)
        
    print(f'{uid} - Saved KeyResp values')
        
        

In [ ]:
n=0
for fname in flist:
    regressors(fname)
    n+=1
print(f'Regressors saved for {n} participants')